In [25]:
from numpy.random import seed
import random
seed(5)
import tensorflow as tf
tf.random.set_seed(3)

In [2]:
import nltk
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path
from scipy import stats

In [5]:
#import data
train_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/train.csv"
train_data_2="/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/train_funlines.csv"
test_data="/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/dev.csv"

#get data with ascending sort by score
train1=pd.read_csv(train_data)
train2=pd.read_csv(train_data_2)
test=pd.read_csv(test_data)

#get label
train_label=list(train1.meanGrade)+list(train2.meanGrade)+list(test_eval_label)
test_label=test.meanGrade

#get orignial news
train_news=list(train1.original)+list(train2.original)+list(test_eval_news)
train_funny_word=list(train1.edit)+list(train2.edit)+list(test_eval_funny_word)
test_news=test.original
test_funny_word=test.edit

In [6]:
#print(test_news[0])
print(train_funny_word[0])
print(train_news[0])

twins
France is ‘ hunting down its citizens who joined <Isis/> ’ without trial in Iraq


In [7]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
tt=nltk.tokenize.regexp.WordPunctTokenizer()
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()

punctuation = '!,;:?"\'.\'/<>'
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip()

#catch the changed word
def find_tag(news):
    p = re.compile(r'[<](.*?)/[>]', re.S)
    return re.findall(p, news)

In [8]:
def preprocess_news(sentence,funny_word):
    final=[]
    num=0
    bad_sign=["’","‘","-"]
    
    for event in sentence:
        lines=[]       
        event=str(event)
        changed_word=find_tag(event)   
        event=removePunctuation(event)
        event=tt.tokenize(event)
        #tokenize and remove Punctuation
        
        for i in event:
            if i not in bad_sign:
                
                if i in changed_word:
                    #add the funny behind the changed word
                    #lines.append(i)
                    lines.append(funny_word[num])
                else:
                    lines.append(i)
                    
        lines=' '.join(lines)
        lines='[CLS]'+' '+lines+' '+'[SEP]'
        final.append(lines)
        num+=1
    return final

In [9]:
#generate my own train data
train_news=preprocess_news(train_news,train_funny_word)
test_news=preprocess_news(test_news,test_funny_word)
print(train_news[0])

[CLS] France is hunting down its citizens who joined twins without trial in Iraq [SEP]


In [8]:
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
import tensorflow as tf
import numpy as np
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
inputs = tokenizer(train_news,padding=True)
test_inputs=tokenizer(test_news,padding=True,return_tensors="tf")
train_inputs=np.array(inputs['input_ids'])
test_inputs=np.array(test_inputs['input_ids'])

Use built-in AlbertsequenceClassification

In [20]:
#inputs_ = tf.compat.v1.placeholder(tf.int64, shape=(None, 37))
input_layer=tf.keras.Input(shape=(37,),dtype='int64')
bert=TFAlbertForSequenceClassification.from_pretrained('albert-base-v2',output_hidden_states=True,num_labels=1)(input_layer)
layer1=bert[0]
flat=tf.keras.layers.Flatten()(layer1)
classfier=tf.keras.layers.Dense(1)(flat)
model=tf.keras.Model(inputs=input_layer,outputs=classfier)
model.summary()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertForSequenceClassification: ['predictions']
- This IS expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['dropout_5', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 37)]              0         
_________________________________________________________________
tf_albert_for_sequence_class ((None, 1), ((None, 37, 7 11684353  
_________________________________________________________________
flatten (Flatten)            (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2         
Total params: 11,684,355
Trainable params: 11,684,355
Non-trainable params: 0
_________________________________________________________________


In [22]:
opt=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.MeanSquaredError()
metric = tf.keras.metrics.MeanSquaredError()
model.compile(optimizer=opt, loss=loss, metrics=[metric])
model_fit = model.fit(train_inputs, np.array(new1), 
                      batch_size=100, epochs=4,
                      validation_data=(test_inputs,np.array(new2))
                   )

Epoch 1/4
179/179 [==============================] - 3791s 21s/step - loss: 0.0440 - mean_squared_error: 0.0440 - val_loss: 0.0391 - val_mean_squared_error: 0.0391
Epoch 2/4
  2/179 [..............................] - ETA: 30:40 - loss: 0.0372 - mean_squared_error: 0.0372

KeyboardInterrupt: 

In [29]:
model.load_weights('./checkpoints/bert_model1')
#flatten_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('dense_12').output)
#predictions = flatten_layer_model.predict(test_inputs[1])
#print(predictions)

In [14]:
test_eval_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/test_eval.csv"

test_eval=pd.read_csv(test_eval_data)
test_eval_label=test_eval.meanGrade

#get orignial news
test_eval_news=test_eval.original
test_eval_funny_word=test_eval.edit


In [15]:
test_eval_news=preprocess_news(test_eval_news,test_eval_funny_word)

In [16]:
test_eval_inputs=tokenizer(test_eval_news,padding=True,return_tensors="tf")
test_eval_inputs=np.array(test_eval_inputs['input_ids'])

In [27]:
#pre=model.predict(np.array(test_eval_inputs))
pres=[]
for i in pre:
    a=float(i)*3
    pres.append(a)
rmse = np.sqrt(np.mean((test_eval_label - pres)**2))
print(rmse)

0.591259278336026


In [53]:
#output result
import pandas as pd
id=test_eval.id
dataframe = pd.DataFrame({'id':id,'pred':pres})
dataframe.to_csv("task-1-output.csv",index=False,sep=',')

In [55]:
print(len(test_eval_inputs))
psr=stats.pearsonr(np.array(pres),np.array(test_eval_label))
print(psr)

3024
(0.25221025314137613, 4.256689175917397e-45)


In [10]:
from transformers import AlbertTokenizer, TFAlbertModel
#import tensorflow as tf
import numpy as np
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2',token_ids_1=None)
inputs_train = tokenizer(train_news,padding=True,return_tensors="pt")
inputs_test = tokenizer(test_news,padding=True,return_tensors="pt")
inputs_train=np.array(inputs_train['input_ids'])
inputs_test=np.array(inputs_test['input_ids'])

In [20]:
#transform score 0-3 to 0-1
new1=[]
new2=[]
for i in train_label:
    a=i/3
    new1.append(a)
for i in test_label:
    new2.append(i/3)

In [12]:
input_layer=tf.keras.Input(shape=(37,),dtype='int64')
bert=TFAlbertModel.from_pretrained('albert-base-v2', return_dict=True)(input_layer)
layer1=bert[1]
classfier=tf.keras.layers.Dense(1,activation='sigmoid')(layer1)
model=tf.keras.Model(inputs=input_layer,outputs=classfier)
model.summary()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 37)]              0         
_________________________________________________________________
tf_albert_model (TFAlbertMod TFBaseModelOutputWithPool 11683584  
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 11,684,353
Trainable params: 11,684,353
Non-trainable params: 0
_________________________________________________________________


In [26]:
opt=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.MeanSquaredError()
metric = tf.keras.metrics.MeanSquaredError()
model.compile(optimizer=opt, loss=loss, metrics=[metric])
model_fit = model.fit(np.array(test_eval_inputs), np.array(new1), 
                      batch_size=100, epochs=2,
                      validation_data=(inputs_test,np.array(new2))
                   )

Epoch 1/2
31/31 [==============================] - 596s 19s/step - loss: 0.0331 - mean_squared_error: 0.0331 - val_loss: 0.0365 - val_mean_squared_error: 0.0365
Epoch 2/2
31/31 [==============================] - 1123s 36s/step - loss: 0.0288 - mean_squared_error: 0.0288 - val_loss: 0.0410 - val_mean_squared_error: 0.0410


In [27]:
pre=model.predict(np.array(test_eval_inputs))
pres=[]
for i in pre:
    a=float(i)*3
    pres.append(a)
rmse = np.sqrt(np.mean((test_eval_label - pres)**2))
print(rmse)

0.47395877753837234


In [19]:
pre=model.predict(np.array(inputs_test))
pres=[]
for i in pre:
    a=float(i)*3
    pres.append(a)
rmse = np.sqrt(np.mean((test_label - pres)**2))
print(rmse)

0.6250030336784045


In [24]:
import pandas as pd
id=test_eval.id
dataframe = pd.DataFrame({'id':id,'pred':pres})
dataframe.to_csv("task-1-output.csv",index=False,sep=',')

In [28]:
model.save_weights('./checkpoints/bert_model6')